# Interroger des documents (RAG)

In [ ]:
%pip install -qU wget openai

Commencez par télécharger le document qui va vous servir de base de connaissance.

In [ ]:
# Download a file
import wget
import os

file_path = "my_document.pdf"
if os.path.exists(file_path):
    os.remove(file_path)
doc_url = "https://www.legifrance.gouv.fr/download/file/rxcTl0H4YnnzLkMLiP4x15qORfLSKk_h8QsSb2xnJ8Y=/JOE_TEXTE"

wget.download(doc_url, out=file_path)

Puis instancier la connexion à l'API Albert.

In [ ]:
# OpenAI client configuration
import requests
from openai import OpenAI

base_url = "https://albert.api.etalab.gouv.fr/v1"
api_key = "YOUR_API_KEY"

client = OpenAI(base_url=base_url, api_key=api_key)

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}

Vous aurez besoin pour la suite d'un modèle de langage ainsi qu'un modèle d'embeddings (pour le RAG). Pour cela vous pouvez appelez le endpoint `/v1/models` pour obtenir la liste des modèles. Les modèles de langage ont le type *text-generation* et les modèles d'embeddings le type *text-embeddings-inference*.

In [ ]:
language_model, embeddings_model = None, None

for model in client.models.list().data:
    if model.type == "text-generation" and language_model is None:
        language_model = model.id
    if model.type == "text-embeddings-inference" and embeddings_model is None:
        embeddings_model = model.id

print(f"language model: {language_model}\nembeddings model: {embeddings_model}")

Enfin pour vous importer le document dans une collection de notre base vectorielle à l'aide du endpoint POST `/v1/files`.

Vous devez spécifier le modèle d'embeddings qui sera utilisé pour vectoriser votre document. Vous pouvez trouver la liste des modèles avec le endpoint `/v1/models`. Les modèles d'embeddings sont indiqués avec le type _feature-extraction_.

Le endpoint POST `/v1/files` doit retourner un status _success_.

In [ ]:
# Remove previous files
collection = "tutorial"
response = session.delete(f"{base_url}/files/{collection}")
response.status_code

In [ ]:
# Upload a file
params = {"collection": collection, "embeddings_model": embeddings_model}

files = {"files": (os.path.basename(file_path), open(file_path, "rb"), "application/pdf")}
response = session.post(f"{base_url}/files", params=params, files=files)

response.json()

Vous pouvez observer les fichiers que vous avez importer dans une collection à l'aide du endpoint GET `/v1/files.`

In [ ]:
# Retrieve the file ID for RAG
response = session.get(f"{base_url}/files/{collection}")
response.json()
file_id = response.json()["data"][0]["id"]
print(file_id)

In [ ]:
prompt = "Qui est Ulrich Tan ?"
data = {"collections": [collection], "model": embeddings_model, "k": 6, "prompt": prompt}
response = session.post(url=f"{base_url}/search", json=data, headers={"Authorization": f"Bearer {api_key}"})

prompt_template = "Réponds à la question suivante en te basant sur les documents ci-dessous : {prompt}\n\nDocuments :\n\n{chunks}"
chunks = "\n\n\n".join([result["chunk"]["content"] for result in response.json()["data"]])
prompt = prompt_template.format(prompt=prompt, chunks=chunks)

response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=language_model,
    stream=False,
    n=1,
)

response = response.choices[0].message.content
print(response)

Vous pouvez également faire ajouter une recherche sur internet en spécifiant "internet" dans la liste des collections.

In [ ]:
prompt = "Qui est Ulrich Tan ?"
collections = [collection, "internet"]
data = {"collections": collections, "model": embeddings_model, "k": 6, "prompt": prompt}
response = session.post(url=f"{base_url}/search", json=data, headers={"Authorization": f"Bearer {api_key}"})

prompt_template = "Réponds à la question suivante en te basant sur les documents ci-dessous : {prompt}\n\nDocuments :\n\n{chunks}"
chunks = "\n\n\n".join([result["chunk"]["content"] for result in response.json()["data"]])
prompt = prompt_template.format(prompt=prompt, chunks=chunks)

response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=language_model,
    stream=False,
    n=1,
)

response = response.choices[0].message.content
print(response)